In [2]:
import numpy as np

In [104]:
def get_triplet_constraints(n, d):
    '''
    Generate all triplets
    '''
    X = np.random.randn(n,d)
    constraints = []
    
    
    x = np.reshape(X, (n*d,1))
    
    for i in range(n):
        for j in range(n):
            for k in range(j):
                if i!=j and i!=k and j!=k:
                    q = [i , j, k]
                    M = np.zeros((n*d,n*d))
                    if np.linalg.norm(X[q[0],:] - X[q[1],:]) < np.linalg.norm(X[q[0],:] - X[q[2],:]):
                        close = q[1]*d
                        far = q[2]*d                      
                    else: 
                        close = q[2]*d
                        far = q[1]*d
                        
                    for D in range(d):
                        M[close+D, close+D] = -1
                        M[far+D, far+D] = 1
                        
                        M[close+D, q[0]*d+D ]= 1
                        M[far + D, q[0]*d+D] = -1
                        M[q[0]*d+D, close+ D ]= 1
                        M[q[0]*d+D, far + D] = -1 
                    constraints.append(M)

    return constraints, x

In [97]:
def generate_trace_zero_constraints(dim, num_constraints):
    constraints = []
    for _ in range(num_constraints):
        M = np.random.randn(dim,dim)
        M = np.matmul(np.transpose(M),M)
        for i in range(dim):
            M[i,i] = 0
        i = np.random.randint(0,dim)
        M[i,i]=1
        j = np.random.randint(0,dim)
        while(i==j):
            j = np.random.randint(0,dim)
        M[j,j]=-1
        constraints.append(M)
    x = np.random.randn(dim,1)

    for i in range(len(constraints)):
        if np.matmul(np.transpose(x),np.matmul(constraints[i],x)) <0:
            constraints[i] = -1*constraints[i]
        elif np.matmul(np.transpose(x),np.matmul(constraints[i],x)) ==0:
            print('Oh no! Something evaluates to 0!')

    return constraints, x

In [98]:
def get_sigmoid_denom(x):
    return 1+np.exp(x)

In [99]:
def get_hessian_gradient_objective(dim, num_constraints):
    hessian = np.zeros((dim,dim))
    grad = np.zeros((dim,1))
    obj = 0
    
    constraints,x = generate_trace_zero_constraints(dim, num_constraints)
    #x = 10*x
    x = np.random.randn(dim,1)
    for constraint in constraints:
        quad_form = np.matmul( np.matmul(np.transpose(x), constraint), x)
        denom = get_sigmoid_denom(quad_form)
        P_x = np.matmul(constraint, x)
        
        obj +=np.log(1+np.exp(-quad_form))
        grad += 2/denom * P_x
        hessian += 4*np.exp(quad_form) / (denom*denom)* np.matmul(P_x, np.transpose(P_x)) + 2/denom*constraint
    return hessian, grad, obj

In [112]:
def get_hessian_gradient_objective_triplet(dim, points):
    hessian = np.zeros((dim*points,dim*points))
    grad = np.zeros((dim*points,1))
    obj = 0
    
    #constraints,x = generate_trace_zero_constraints(dim, num_constraints)
    constraints,x = get_triplet_constraints(points, dim)
    x = 10*x
    print()
    #x = np.random.randn(dim*points,1)
    for constraint in constraints:
        quad_form = np.matmul( np.matmul(np.transpose(x), constraint), x)
        denom = get_sigmoid_denom(quad_form)
        P_x = np.matmul(constraint, x)
        
        obj +=np.log(1+np.exp(-quad_form))
        grad += 2/denom * P_x
        hessian += 4*np.exp(quad_form) / (denom*denom)* np.matmul(P_x, np.transpose(P_x)) + 2/denom*constraint
    return hessian, grad, obj

In [106]:
def check_eigvalues(x):
    flag_pos = 0
    flag_neg = 0
    category = ''
    for i in range(len(x)):
        if x[i]>0:
            flag_pos = 1
        elif x[i]<0:
            flag_neg = 1
    if flag_pos ==1 and flag_neg ==1:
        category = 'indefinite'
    elif flag_neg ==0 and flag_pos == 1:
        category = 'PSD'
    else:
        category = 'NSD'
    print(category)
    return category

In [113]:
trials = 100
psd = 0
nsd = 0
indef = 0
for _ in range(trials):
    hessian, grad, obj = get_hessian_gradient_objective_triplet(2, 10)
    print('gradient',np.linalg.norm(grad), 'objective',obj)
    eig_val = np.linalg.eig(hessian)[0]
    print(eig_val)
    category = check_eigvalues(eig_val)
    if category == 'indefinite':
        indef +=1
    elif category =='PSD':
        psd +=1
    else:
        nsd +=1
            
print('proportion indef', indef / float(trials))
print('proportion psd', psd / float(trials))
print('proportion nsd', nsd / float(trials))


gradient 18.8774122002 objective [[ 0.60895325]]


/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: overflow encountered in multiply
/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in true_divide


LinAlgError: Array must not contain infs or NaNs

In [38]:
X = np.random.randn(3,2)

In [39]:
print(X)

[[-0.23182879  1.16931282]
 [ 0.66772429 -0.47582168]
 [ 0.54905949 -0.22580684]]


In [56]:
test= np.reshape(X, (6,1))

In [58]:
print(test)

[[-0.23182879]
 [ 1.16931282]
 [ 0.66772429]
 [-0.47582168]
 [ 0.54905949]
 [-0.22580684]]
